# Misison Statement Embedding Pipeline

Basic Pipeline to create document embedding vectors for all the mission statements using Glove 

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import os
import pdb

#NLP PACKAGES
import gensim
from gensim.corpora import Dictionary
from gensim.models.tfidfmodel import TfidfModel
from gensim.matutils import sparse2full
import numpy as np
np.random.seed(400)
import spacy
nlp  = spacy.load('en_core_web_md')



%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
# Custom functions
from nlp_functions import preprocess_spacy, preprocess_docs,doc_embed_charity_notfidf

## 0.0 Load Data

In [3]:
project_dir =  os.path.abspath(os.path.join(os.getcwd() ,"../"))
file_name = os.path.join(project_dir,'data','processed','charity_data_cleaned.csv')
all_charity = pd.read_csv(file_name)

## 1.0 Pre-process raw text

In [4]:
## Extract mission statements texts (w/ their named dropped from the statement) as a list of strings
mission_text = all_charity['description_noname'].astype('str')

#Preprocess mission descriptions
mission_text_pre = preprocess_docs(mission_text)

# 2.0 Embed Docusments

In [6]:
#Set min and max # of words to filter. 
#Min n excludes all words that only have n instances in the whole corpus of text
#Max p excludes all words that appear in more than p% of documents 
word_min = 0
word_max_perc = .5
charity_docs_emb, charity_docs_dict, charity_emb_vecs = doc_embed_charity_notfidf(mission_text_pre,
                                                                                 word_min=word_min, 
                                                                                word_max_perc=word_max_perc)




Total # of docs: 5637
Total # of words in dict: 11631


### Document Embedding  Model Outputs

* charity_docs_emb = matrix of document embedding for each charity (N charities x N glove dimensions[300])
* charity_docs_dict = dictionary of words in all mission statements (that werent filtered out) 
* charity_emb_vecs = glove vectors for all the words in the dictions (N words x N glove dimensions[300]) 

In [149]:
#Store output in dictionary and save

#Charity docs emb
charity_model_dict = {'charity_docs_emb': charity_docs_emb, 
                      'charity_docs_dict': charity_docs_dict,
                      'charity_emb_vecs': charity_emb_vecs}

save_dir = os.path.join(project_dir,'models','charity')
with open(os.path.join(save_dir,'charity_model_min_{}_max_{}_notfidf.pickle'.format(word_min,word_max_perc)), 'wb') as handle:
    pickle.dump(charity_model_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)